In [ ]:
#  import libraries
import pandas as pd
import numpy as np

In [ ]:
# read dataset
if __name__ == '__main__':
    from google.colab import drive
    drive.mount('/content/drive')

    YourFilePath = 'pa1'

    train_features = pd.read_csv('/content/drive/MyDrive/'+YourFilePath+'/train_features.csv')
    test_features = pd.read_csv('/content/drive/MyDrive/'+YourFilePath+'/test_features.csv')
    train_labels = pd.read_csv('/content/drive/MyDrive/'+YourFilePath+'/train_labels.csv')
    test_labels = pd.read_csv('/content/drive/MyDrive/'+YourFilePath+'/test_labels.csv')

Mounted at /content/drive


In [ ]:
if __name__ == '__main__':
  print(train_features.describe())

          Elevation        Aspect         Slope  \
count  12096.000000  12096.000000  12096.000000   
mean    2748.303406    156.311177     16.510499   
std      417.822995    109.903370      8.471222   
min     1863.000000      0.000000      0.000000   
25%     2374.000000     65.000000     10.000000   
50%     2751.000000    125.000000     15.000000   
75%     3102.000000    259.000000     22.000000   
max     3849.000000    360.000000     50.000000   

       Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology  \
count                      12096.000000                    12096.000000   
mean                         227.905754                       51.165923   
std                          210.565445                       61.547238   
min                            0.000000                     -146.000000   
25%                           67.000000                        5.000000   
50%                          180.000000                       32.000000   
75%            

In [ ]:
# Data Preprocessing
def z_score_normalization(input_array):
  # input_array: numpy array of shape (num_rows, num_features)
  # todo start #
  normalized_array = (input_array - np.mean(input_array,axis = 0)) / np.std(input_array,axis = 0)
  # todo end #
  return normalized_array

In [ ]:
 # Manhattan distance
def manhattan_distance(X_train, X_test):
  # X_train: numpy array of shape (num_rows_train, num_features)
  # X_test: numpy array of shape (num_rows_test, num_features)
  # todo start #
  X_test = np.expand_dims(X_test,1)
  distance = np.sum(np.abs(X_train - X_test),axis = 2)
  # todo end #
  return distance
  # distance: numpy array of shape (num_rows_test, num_rows_train)

# Cosine distance
def cosine_distance(X_train, X_test):
  # X_train: numpy array of shape (num_rows_train, num_features)
  # X_test: numpy array of shape (num_rows_test, num_features)
  # todo start #
  numerator = np.sum(np.expand_dims(X_test,axis =1) * X_train,axis = 2)
  denominator = np.expand_dims(np.sqrt(np.sum(np.square(X_test),axis = 1)),axis = 1) * (np.sqrt(np.sum(np.square(X_train),axis = 1)))
  fraction = numerator / denominator
  distance = 1 - fraction
  # todo end #
  return distance
  # distance: numpy array of shape (num_rows_test, num_rows_train)

In [ ]:
# Classification based on the Nearest Neighbors
def knn_prediction(distances, y_train, k):
  # distance: numpy array of shape (num_rows_test, num_rows_train), return value from previous distance functions
  # y_train: numpy array of shape (num_rows_train, ),  the labels of training data
  # k: integer, k in "K-nearest neighbors"

  # todo start #
  sort_distance = np.argsort(distances,axis = 1)
  y_label = np.take(y_train,sort_distance)[:,:k]
  mask = (np.arange(np.max(y_train)+1).reshape(1,1,np.max(y_train)+1) == np.expand_dims(y_label,axis = 2))
  count_of_dataPoint = np.sum(mask.astype(int),axis = 1) # total number of each label
  prediction = np.argmax(count_of_dataPoint,axis = 1) # Majority voting

  #Find tie
  tie = (count_of_dataPoint == np.expand_dims(count_of_dataPoint.max(axis = 1),axis = 1))
  tie_index = np.where(np.sum(tie,axis = 1) > 1)

  #inverse distance
  sort_inverse_distance = np.sort(distances,axis = 1)[:,:k]
  inverse_distance = 1 / sort_inverse_distance
  t = mask * np.expand_dims(inverse_distance,2)
  t1 = np.sum(t,axis = 1)
  inverse = np.argmax(t1,axis = 1)
  prediction[tie_index] = inverse[tie_index]
  y_classes = np.arange(np.max(y_train))+1
  return prediction,y_classes
  # prediction: 1-D numpy array of shape (num_rows_test, )

In [ ]:
# Evaluation of the K Nearest Neighbors Classifier
def f_score(y_test, prediction, y_classes):
  num_classes = np.arange(len(y_classes))+1

  # todo start #
  f_score_array = np.zeros(len(y_classes))
  for i in num_classes:
    # Fill in the missing code/s #
    TP = np.sum( (prediction == i) & (y_test.flatten() == i))
    FP = np.sum( (prediction == i) & (y_test.flatten() != i))
    FN = np.sum( (prediction != i) & (y_test.flatten() == i))

    precision = TP / (FP+ TP+ np.finfo(float).eps)
    recall = TP / (TP + FN)
    f1_score = (2 * precision * recall) / (precision + recall )
    f_score_array[i-1] = f1_score
    # todo end #

  return f_score_array
  # f_score: 1-D array with shape (num_classes, )


In [ ]:
# Euclidean Distance Calculation
def centroid_euclidean_distance(X_train, centroids):
  # X_train: numpy array of shape (num_rows_train, num_features)
  # centroids: numpy array of shape (num_clusters, num_features)
  # todo start #
  X_train = np.expand_dims(X_train,1)
  distance = np.sqrt(np.sum(np.square(X_train - centroids),axis = 2)).T
  # todo end #
  return distance
  # distance: numpy array of shape (num_clusters, num_rows_train)

#Cluster Assignment
def cluster_assignment(distance):
  # distance: numpy array of shape (num_clusters, num_rows_train)
  # todo start #
  assignments = np.argmin(distance,axis = 0)
  #assignments = distance[min,np.arange(distance.shape[1])]
  # todo end #
  return assignments
  # assignment: 1-D numpy array of shape (num_rows_train, )

# Calculate New Centroids
def calculate_centroids(X_train, assignment, k):
  # X_train: numpy array of shape (num_rows_train, num_features)
  # assignment: 1-D numpy array of shape (num_rows_train, )
  # k: a scalar value for the number of clusters (NOTE: Include empty clusters in counting k)

  conut = np.bincount(assignment.astype(int)).reshape(k,1)
  conut = conut.astype(float)
  label = np.arange(k).reshape(k,1)
  dup = np.tile(assignment.astype(int),(k,1))
  mask = (dup[:,:] == label[:])
  mask = mask.astype(float)
  new_centroids = (mask @ X_train) / conut

  return new_centroids
  # new_centroids: numpy array of shape (num_clusters, num_features)

In [ ]:
#Stop When There Are No More Cluster Reassignments
def k_means_cluster_reassignment(X_train, initial_centroids, max_iterations=100):
  # X_train: numpy array of shape (num_rows_train, num_features)
  # initial_centroids: numpy array of shape (num_clusters, num_features)
  # max_terations: the maximum number of iterations for refining the model
  centroids = initial_centroids
  k = centroids.shape[0]
  prev_assignment = None
  for iteration in range(max_iterations):
    distance = centroid_euclidean_distance(X_train, centroids)
    assignment = cluster_assignment(distance)
    centroids = calculate_centroids(X_train, assignment, k)
    if np.array_equal(assignment, prev_assignment):
      break  # Stop if there are no more cluster reassignments

    prev_assignment = assignment
  return assignment, centroids
  # centroids: numpy array of shape (num_clusters, num_features)

# Stop When Centroid Change is Below Threshold
def k_means_centroid_value(X_train, initial_centroids, max_iterations=100, threshold_value=0.0001):
  # X_train: numpy array of shape (num_rows_train, num_features)
  # initial_centroids: numpy array of shape (num_clusters, num_features)
  # max_terations: the maximum number of iterations for refining the model
  # threshold_value: a scaler value for the allowable difference between iterations
  centroids = initial_centroids
  k = centroids.shape[0]
  prev_centroids = None
  for iteration in range(max_iterations):
    distance = centroid_euclidean_distance(X_train, centroids)
    assignment = cluster_assignment(distance)
    centroids = calculate_centroids(X_train, assignment, k)
    # todo start #
    if prev_centroids is not None and np.all(np.abs(centroids - prev_centroids) < threshold_value):
      break  # Stop if the centroid values have converged

    prev_centroids = centroids
  return assignment, centroids
  # centroids: numpy array of shape (num_clusters, num_features)

In [ ]:
# Evaluating the value of k
def silhouette_score_for_k(X_train, assignments):
  n_samples = len(X_train)
  silhouette_scores = []

  for i in range(n_samples):
      point = X_train[i]
      assignment = assignments[i]
      same_cluster_distances = np.sqrt(np.sum((X_train[assignments == assignment] - point) ** 2, axis=1))
      same_cluster_distances = same_cluster_distances[same_cluster_distances != 0]


      other_cluster_distances = []
      for j in range(len(X_train)):
          if assignments[j] != assignment:
              other_cluster_distance = np.mean(np.sqrt(np.sum((X_train[assignments == assignments[j]] - point) ** 2, axis=1)))
              other_cluster_distances.append(other_cluster_distance)

      if len(other_cluster_distances) == 0:
          silhouette_score = 0
      else:
          min_other_cluster_distance = np.min(other_cluster_distances)
          silhouette_score = (min_other_cluster_distance - np.mean(same_cluster_distances)) / max(min_other_cluster_distance, np.mean(same_cluster_distances))

      silhouette_scores.append(silhouette_score)
      silhouette_coef = np.mean(silhouette_scores)
  return silhouette_coef
  # silhouette_coef: a scalar value

In [ ]:
if __name__ == '__main__':
  X_train = np.array(train_features)
  X_test = np.array(test_features)
  y_train = np.array(train_labels)
  y_test = np.array(test_labels)
  np.random.seed(0)
  n_clusters=3
  initial_centroid = np.random.rand(n_clusters, X_train.shape[1])
  n_neighbors=5

  # Task 1: Data Preprocessing
  X_train = z_score_normalization(X_train)
  X_test = z_score_normalization(X_test)
  print("The normalized features of the first 5 samples of X_train are: ", X_train[:5, :])

  # Task 2.1: Manhattan distance
  m_distance = manhattan_distance(X_train, X_test)
  print('The manhattan distance between the first 5 X_train and first 5 X_test are', m_distance[:5, :5])

  # Task 2.2: Cosine distance
  c_distance = cosine_distance(X_train, X_test)
  print('The cosine distance between the first 5 X_train and first 5 X_test are', c_distance[:5, :5])

  # Task 3: Classification based on the Nearest Neighbors
  prediction, y_classes = knn_prediction(m_distance, y_train, n_neighbors)
  print('The predicted classes for the first 10 samples in X_test are ', prediction[:10])
  print('The true classes of these samples are ', y_test[:10].flatten())

  # Task 4: Evaluation of the K Nearest Neighbors Classifier
  f_score_array = f_score(y_test.flatten(), prediction, y_classes)
  print('The F-scores for each class are ', f_score_array)

  # Task 5-6: K-Means Clustering
  assignment, centroid = k_means_cluster_reassignment(X_train, initial_centroid, max_iterations=100)
  print('With cluster reassignment as the stopping criteria, the assignments for the first 5 samples of X_train are: ', assignment[:5])
  assignment, centroid = k_means_centroid_value(X_train, initial_centroid, max_iterations=100)
  print('With centroid value as the stopping criteria, the assignments for the first 5 samples of X_train are: ', assignment[:5])

  # Task 6.3: Evaluating the value of k
  silhouette_avg = silhouette_score_for_k(X_train[:300, :], assignment[:300])
  print('The silhouette coefficient is ', silhouette_avg)

The normalized features of the first 5 samples of X_train are:  [[-3.64531641e-01 -9.58255697e-01 -1.59493603e+00  1.42927013e-01
  -8.31362047e-01 -9.05389383e-01  2.66760906e-01  5.75366176e-01
   2.87038390e-01  4.35034917e+00  7.80187617e-01]
 [-3.78892383e-01 -9.12759306e-01 -1.71298762e+00 -7.55414141e-02
  -9.28852173e-01 -9.95624417e-01  2.33971167e-01  7.07024400e-01
   3.52285503e-01  4.30109144e+00  7.80187617e-01]
 [ 1.33307398e-01 -1.57519217e-01 -8.86626443e-01  1.90420149e-01
   2.24780993e-01  1.10234013e+00  6.93027524e-01  8.38682623e-01
   4.30090096e-03  4.20622470e+00 -5.66064681e-01]
 [-3.66925098e-01 -1.01285137e+00 -1.71298762e+00 -3.55750918e-01
  -8.47610401e-01 -9.94872458e-01  2.33971167e-01  6.63138325e-01
   3.30536465e-01  4.25274589e+00  7.80187617e-01]
 [-4.05220409e-01 -2.21214165e-01 -1.24078123e+00  3.42398185e-01
  -1.07508736e+00 -1.23850705e+00  5.61868565e-01  7.94796549e-01
   1.13046089e-01  4.12412848e+00  7.80187617e-01]]
The manhattan distan